# DATA PROCESSING FOR XGB AUTO-REGRESSION

In [1]:
import pandas as pd 
import xgboost as xgb
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from calendar import monthrange
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('ais_train.csv', sep='|')
test = pd.read_csv('ais_test.csv')

In [3]:
train.shape

(1522065, 11)

In [4]:
test.shape

(51739, 4)

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# Functions needed in preprocessing

## Add a flag if there's a probability of looking at a new trip 

In [5]:
def add_new_trip_flag(df):
    # Selecting a threshold where we tell the model to consider the posibility of a new trip starting
    # Current: 8h20m, 200km traveled (15mph avg speed)
    # Not realistic but reasonable to consider shorter path for training
    # Maybe change later
    df['new_trip'] = np.where(
        (df['time_diff_minutes'] == 0) | (df['time_diff_minutes'] >= 500) |
        (df['distance_km'].isna()) | (df['distance_km'] >= 200),
        1, 0  # Set to 1 if conditions are met, else 0
    )
    # Might add differerence in eta as an indicator
    return df

## Time processing

In [6]:
def process_time(df):
    # Separate Date-Time in single attributes
    df['year_rec'] = df['time'].dt.year
    df['month_rec'] = df['time'].dt.month
    df['day_rec'] = df['time'].dt.day
    df['hour_rec'] = df['time'].dt.hour
    df['minute_rec'] = df['time'].dt.minute
    df['dayofweek_rec'] = df['time'].dt.dayofweek


    df['year_eta'] = df['etaStd'].dt.year.fillna(0).astype('int32')
    df['month_eta'] = df['etaStd'].dt.month.fillna(0).astype('int32')
    df['day_eta'] = df['etaStd'].dt.day.fillna(0).astype('int32')
    df['hour_eta'] = df['etaStd'].dt.hour.fillna(0).astype('int32')
    df['minute_eta'] = df['etaStd'].dt.minute.fillna(0).astype('int32')
    df['dayofweek_eta'] = df['etaStd'].dt.dayofweek

    df['etaStd_seq'] = df['etaStd'].astype(int) / 10**9  # Converts to seconds
    df['time_seq'] = df['time'].astype(int) / 10**9  # Converts to seconds
    return df

### Use distance and time to get avg speed

In [7]:
def calculate_speed_last_stretch(df):
    df['mph_last_stretch'] = (df['distance_km'] *  0.621371) / (df['time_diff_minutes'] / 60)

    # Remove unrealistic values over the datadase
    df = df[(df['mph_last_stretch']<60) | (df['mph_last_stretch'].isna())] 
    
    return df 

### Time since last data collection

In [8]:
def calculate_time_difference(df):
    # Make sure is sorted
    df = df.sort_values(by=['vesselId', 'time'])
    
    # Calculate time difference
    df['time_diff_minutes'] = df.groupby('vesselId')['time'].diff().dt.total_seconds() / 60
    # Make integer changing numbers 0<x<1 to one so that the 0 is only a separator between ships
    df['time_diff_minutes'] = df['time_diff_minutes'].fillna(0).apply(lambda x: 1 if 0 < x < 1 else x).astype(int)
    
    return df

### Formula for distance on a sphere

In [9]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


### Calculate distances using Haversine

In [10]:
def calculate_distances(df):
    # Make sure is sorted
    df = df.sort_values(by=['vesselId', 'time'])

    # Create new columns for previous latitude and longitude
    df['prev_latitude'] = df.groupby('vesselId')['latitude'].shift(1)
    df['prev_longitude'] = df.groupby('vesselId')['longitude'].shift(1)

    # Calculate distance between current and previous position
    df['distance_km'] = df.apply(
        lambda row: haversine(row['latitude'], row['longitude'], 
                               row['prev_latitude'], row['prev_longitude']) 
                     if pd.notna(row['prev_latitude']) and pd.notna(row['prev_longitude']) 
                     else np.nan, axis=1)
    
    # Drop the helper columns if not needed
    df.drop(columns=['prev_latitude', 'prev_longitude'], inplace=True)

    return df

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# Preprocessing function

In [11]:
def preprocess_all(df):
    # Replacing default with Nan bacause too close to valid values, eliminate non valid values
    df['cog'] = df['cog'].replace(360, np.nan)
    df = df[(df['cog'] <= 360) | (df['cog'].isna())]

    # Replacing default with Nan bacause too close to valid values
    df['sog'] = df['sog'].replace(1023, np.nan)

    # Replacing default with Nan bacause too close to valid values
    # Changing uncertain values to bigger number to be further away from sample pool
    # Adding uncertainty flag
    df['rot'] = df['rot'].replace(128, np.nan)
    df['rot'] = df['rot'].replace({127: 200, -127: -200})
    df['uncertain_rot'] = np.where(df['rot'].isin([200, -200]), 1, 0)

    # Replacing default value with NaN to not get taken in consideration by regression
    df['heading'] = df['heading'].replace(511, np.nan)

    # One hot encoding for the navigation status
    df = pd.get_dummies(df, columns=['navstat'], drop_first = True)

    # Get Date-Time from standard and raw data
    df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC')
    df['etaRaw'] = df['etaRaw'].fillna(0)
    df['etaRaw'] = df['etaRaw'].apply(lambda x: f"{2024}-{x}")
    df['etaRaw'] = pd.to_datetime(df['etaRaw'], errors='coerce').dt.tz_localize('UTC')
    df.rename(columns={'etaRaw': 'etaStd'}, inplace=True)

    # Handle first month of the years ETA year to be 2023
    df['etaStd'] = df.apply(lambda row: row['etaStd'].replace(year=row['etaStd'].year - 1)
                            if row['etaStd'].month in [11, 12] and row['time'].month in [1, 2] 
                            else row['etaStd'], axis=1) 
    
    df = process_time(df)
    
    # FEATURE ENGINEERING
    df = calculate_distances(df)
    df = calculate_time_difference(df)
    df = calculate_speed_last_stretch(df)
    df = add_new_trip_flag(df)
    
    
    return df

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# Lag features and moving average

In [12]:
def add_lag_features(df, features_to_lag, steps=1):
    
    for step in range(1, steps + 1):
            for feature in features_to_lag:
                lag_col = f'{feature}_lag{step}'
                df[lag_col] = df.groupby('vesselId')[feature].shift(step)
        
    return df

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# Encode labels for training

In [13]:
def encode_labels(df):
    label_encoder = LabelEncoder()
    df['vesselId_encoded'] = label_encoder.fit_transform(df['vesselId'])
    df['portId_encoded'] = label_encoder.fit_transform(df['portId'])
    #df['shippingLineId_encoded'] = label_encoder.fit_transform(df['shippingLineId'])

    # Drop non valid columns
    df.drop(columns=['vesselId'], inplace=True)
    df.drop(columns=['portId'], inplace=True)
    #df.drop(columns=['shippingLineId'], inplace=True)
    df.drop(columns=['time'], inplace=True)
    df.drop(columns=['etaStd'], inplace=True)
    
    return df

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# Update row function
### Used to update row before prediction to add lag features of predicted values

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# DATA PROCESSING

In [14]:
train['source'] = 'train'
test['source'] = 'test'

comb = pd.concat([train, test], axis=0, ignore_index=True)
comb = comb.sort_values(by=['vesselId', 'time'])
comb.tail(1000)

time    cog   sog   rot  heading  navstat  \
1368857  2024-04-25 06:57:52  100.7  16.5   3.0    100.0      0.0   
1369030  2024-04-25 07:18:09  102.2  16.7   3.0    101.0      0.0   
1369210  2024-04-25 07:38:40  150.1  15.9   3.0    150.0      0.0   
1369386  2024-04-25 07:59:17  249.1   0.3  -8.0    143.0      0.0   
1369517  2024-04-25 08:19:27    0.0   0.0  -1.0    134.0      5.0   
1369636  2024-04-25 08:37:27    0.0   0.0  -2.0    134.0      5.0   
1369786  2024-04-25 08:58:27    0.0   0.0  -3.0    134.0      5.0   
1369947  2024-04-25 09:13:28    0.0   0.0  -1.0    134.0      5.0   
1370130  2024-04-25 09:34:28    0.0   0.0  -1.0    134.0      5.0   
1370246  2024-04-25 09:52:29    0.0   0.0  -2.0    135.0      5.0   
1370417  2024-04-25 10:13:28    0.0   0.0  -2.0    134.0      5.0   
1370607  2024-04-25 10:34:29    0.0   0.0   0.0    134.0      5.0   
1370786  2024-04-25 10:55:29    0.0   0.0   2.0    134.0      5.0   
1370959  2024-04-25 11:16:29    0.0   0.0  -2.0    134.0      5.0   
1371153  2024-04-25 11:37:29    0.0   0.0   1.0    134.0      5.0   
1371250  2024-04-25 11:55:29    0.0   0.0   2.0    134.0      5.0   
1371434  2024-04-25 12:16:29    0.0   0.0  -1.0    134.0      5.0   
1371622  2024-04-25 12:37:30    0.0   0.0  -1.0    134.0      5.0   
1371798  2024-04-25 12:58:30    0.0   0.0  -1.0    135.0      5.0   
1371946  2024-04-25 13:13:30    0.0   0.0   2.0    135.0      5.0   
1372111  2024-04-25 13:34:30    0.0   0.0   1.0    134.0      5.0   
1372292  2024-04-25 13:55:31    0.0   0.0  -2.0    134.0      5.0   
1372388  2024-04-25 14:13:31    0.0   0.0  -3.0    134.0      5.0   
1372661  2024-04-25 14:36:44  330.1  16.8   7.0    329.0      0.0   
1372764  2024-04-25 14:56:45  294.0  19.6   5.0    293.0      0.0   
1372833  2024-04-25 15:07:51  282.2  20.3  -3.0    282.0      0.0   
1373041  2024-04-25 15:36:27  277.7  13.7  -3.0    277.0      0.0   
1373144  2024-04-25 15:49:33  280.4  14.0  -7.0    279.0      0.0   
1373298  2024-04-25 16:06:06  296.8  19.2  15.0    299.0      0.0   
1380852  2024-04-26 07:32:53   77.4  18.4  -3.0     77.0      0.0   
1381082  2024-04-26 07:55:22   87.6  18.2   7.0     88.0      0.0   
1381283  2024-04-26 08:16:16  134.7  17.7   6.0    136.0      0.0   
1381439  2024-04-26 08:37:11  136.9  18.0   8.0    136.0      0.0   
1381621  2024-04-26 08:57:47  131.5  17.8   2.0    129.0      0.0   
1381767  2024-04-26 09:18:04  136.8  17.6   0.0    135.0      0.0   
1381942  2024-04-26 09:38:17  141.2  17.7  -4.0    139.0      0.0   
1382125  2024-04-26 09:58:59  160.7  17.8  -4.0    160.0      0.0   
1382275  2024-04-26 10:19:17  160.0  18.0  -9.0    160.0      0.0   
1382439  2024-04-26 10:39:47  160.2  17.9  12.0    160.0      0.0   
1382606  2024-04-26 10:59:52  160.9  17.8  -2.0    161.0      0.0   
1382650  2024-04-26 11:09:47  160.3  17.9  -7.0    160.0      0.0   
1382789  2024-04-26 11:16:52  160.2  17.9   3.0    160.0      0.0   
1383978  2024-04-26 13:57:29  143.5  17.0   7.0    142.0      0.0   
1384245  2024-04-26 14:14:53  143.7  17.4   3.0    142.0      0.0   
1384353  2024-04-26 14:34:53  128.7  17.1  -7.0    128.0      0.0   
1384544  2024-04-26 14:56:17  127.8  15.7   1.0    128.0      0.0   
1384690  2024-04-26 15:16:35  154.4  15.3  -1.0    155.0      0.0   
1384857  2024-04-26 15:37:17  157.7  17.4   3.0    157.0      0.0   
1385004  2024-04-26 15:58:05  170.3  16.4   2.0    169.0      0.0   
1385179  2024-04-26 16:18:17  198.8  15.5  -8.0    197.0      0.0   
1385345  2024-04-26 16:38:59  175.5  14.9  -3.0    175.0      0.0   
1385511  2024-04-26 16:59:46  174.4  10.5   7.0    173.0      0.0   
1385647  2024-04-26 17:14:47  173.8  10.9   2.0    173.0      0.0   
1385806  2024-04-26 17:34:47  189.6  12.7  -2.0    190.0      0.0   
1386010  2024-04-26 17:55:17  198.1  13.3   3.0    198.0      0.0   
1386132  2024-04-26 18:15:48  190.4  13.4   3.0    190.0      0.0   
1386324  2024-04-26 18:36:47  112.0  13.3   8.0    113.0      0.0   
1386472  2024-04-26 18

In [15]:
comb = preprocess_all(comb)

In [16]:
# Set lag features an lag steps
features_to_lag = ['time_diff_minutes', 'latitude', 'longitude', 'mph_last_stretch', 'distance_km']
steps = 2

comb = add_lag_features(comb, features_to_lag, steps)

In [17]:
comb[['time', 'year_rec', 'month_rec', 'day_rec', 'hour_rec', 'minute_rec']].sort_values('time').head()

time  year_rec  month_rec  day_rec  hour_rec  \
0 2024-01-01 00:00:25+00:00      2024          1        1         0   
1 2024-01-01 00:00:36+00:00      2024          1        1         0   
2 2024-01-01 00:01:45+00:00      2024          1        1         0   
3 2024-01-01 00:03:11+00:00      2024          1        1         0   
4 2024-01-01 00:03:51+00:00      2024          1        1         0   

   minute_rec  
0           0  
1           0  
2           1  
3           3  
4           3

In [18]:
comb = encode_labels(comb)
comb.head()

cog   sog  rot  heading  latitude  longitude source  ID  \
131115  308.1  17.1 -6.0    316.0   7.50361   77.58340  train NaN   
131279  307.6  17.3  5.0    313.0   7.57302   77.49505  train NaN   
131514  306.8  16.9  5.0    312.0   7.65043   77.39404  train NaN   
131696  307.9  16.9  6.0    313.0   7.71275   77.31394  train NaN   
131885  307.0  16.3  7.0    313.0   7.77191   77.23585  train NaN   

        scaling_factor  uncertain_rot  navstat_1.0  navstat_2.0  navstat_3.0  \
131115             NaN              0        False        False        False   
131279             NaN              0        False        False        False   
131514             NaN              0        False        False        False   
131696             NaN              0        False        False        False   
131885             NaN              0        False        False        False   

        navstat_4.0  navstat_5.0  navstat_6.0  navstat_7.0  navstat_8.0  \
131115        False        False        False        False        False   
131279        False        False        False        False        False   
131514        False        False        False        False        False   
131696        False        False        False        False        False   
131885        False        False        False        False        False   

        navstat_9.0  navstat_11.0  navstat_12.0  navstat_13.0  navstat_14.0  \
131115        False         False         False         False         False   
131279        False         False         False         False         False   
131514        False         False         False         False         False   
131696        False         False         False         False         False   
131885        False         False         False         False         False   

        navstat_15.0  year_rec  month_rec  day_rec  hour_rec  minute_rec  \
131115         False      2024          1       12        14           7   
131279         False      2024          1       12        14          31   
131514         False      2024          1       12        14          57   
131696         False      2024          1       12        15          18   
131885         False      2024          1       12        15          39   

        dayofweek_rec  year_eta  month_eta  day_eta  hour_eta  minute_eta  \
131115              4      2024          1        8         6           0   
131279              4      2024          1       14        23          30   
131514              4      2024          1       14        23          30   
131696              4      2024          1       14        23          30   
131885              4      2024          1       14        23          30   

        dayofweek_eta    etaStd_seq      time_seq  distance_km  \
131115            0.0  1.704694e+09  1.705068e+09          NaN   
131279            6.0  1.705275e+09  1.705070e+09    12.426563   
131514            6.0  1.705275e+09  1.705071e+09    14.072336   
131696            6.0  1.705275e+09  1.705073e+09    11.221963   
131885            6.0  1.705275e+09  1.705074e+09    10.830682   

        time_diff_minutes  mph_last_stretch  new_trip  time_diff_minutes_lag1  \
131115                  0               NaN         1                     NaN   
131279                 23         20.143059         0                     0.0   
131514                 26         20.178789         0                    23.0   
131696                 21         19.922864         0                    26.0   
131885                 20         20.189616         0                    21.0   

        latitude_lag1  longitude_lag1  mph_last_stretch_lag1  \
131115            NaN             NaN                    NaN   
131279        7.50361        77.58340                    NaN   
131514        7.57302        77.49505              20.143059   
131696        7.65043        77.39404              20.178789   
131885        7.71275        77.31394              19.922864   


In [19]:
comb[['source',
    'time_seq', 'distance_km', 'time_diff_minutes', 'mph_last_stretch', 
    'time_diff_minutes_lag1', 'latitude_lag1', 'longitude_lag1', 'mph_last_stretch_lag1', 'distance_km_lag1',
    'time_diff_minutes_lag2', 'latitude_lag2', 'longitude_lag2', 'mph_last_stretch_lag2', 'distance_km_lag2',
    'latitude', 'longitude'
     ]].tail(400)


source      time_seq  distance_km  time_diff_minutes  \
1499324  train  1.714965e+09    11.545204                 20   
1499500  train  1.714967e+09    11.598101                 20   
1499680  train  1.714968e+09    11.650856                 21   
1499859  train  1.714969e+09    11.880279                 21   
1500032  train  1.714970e+09    11.088698                 20   
1500043  train  1.714971e+09     3.675888                  6   
1501849  train  1.714984e+09   127.991674                228   
1502078  train  1.714986e+09    14.656049                 26   
1502166  train  1.714987e+09     6.825367                 12   
1502375  train  1.714988e+09    12.433433                 22   
1502671  train  1.714990e+09    21.976231                 40   
1502799  train  1.714991e+09     7.746080                 14   
1503075  train  1.714993e+09    16.256042                 29   
1503242  train  1.714994e+09    11.309273                 20   
1503391  train  1.714995e+09    11.181986                 20   
1503561  train  1.714997e+09    11.510306                 20   
1503711  train  1.714998e+09    11.425280                 20   
1503877  train  1.714999e+09    11.353686                 20   
1504031  train  1.715000e+09    11.353212                 20   
1504188  train  1.715001e+09     8.277632                 15   
1504368  train  1.715003e+09    11.051987                 20   
1504540  train  1.715004e+09    11.197601                 20   
1504555  train  1.715004e+09     6.382032                 11   
1504721  train  1.715005e+09     8.404548                 15   
1504895  train  1.715007e+09    12.709659                 23   
1511072  train  1.715052e+09   401.687510                751   
1511091  train  1.715053e+09     8.452007                 15   
1511328  train  1.715054e+09    14.323682                 25   
1511496  train  1.715055e+09    11.224444                 20   
1511554  train  1.715056e+09     6.185033                 11   
1511839  train  1.715058e+09    16.669268                 29   
1511876  train  1.715059e+09     9.476656                 16   
1513080  train  1.715068e+09    85.999643                157   
1513351  train  1.715070e+09    12.941190                 23   
1513539  train  1.715071e+09     9.183568                 20   
1513684  train  1.715072e+09     2.675479                 20   
1513859  train  1.715074e+09     0.192330                 20   
1513924  train  1.715075e+09     0.002496                 18   
1514083  train  1.715076e+09     0.002295                 15   
1514271  train  1.715077e+09     0.002033                 21   
1514477  train  1.715078e+09     0.002295                 21   
1514708  train  1.715079e+09     0.002033                 21   
1514771  train  1.715080e+09     0.002269                 18   
1514958  train  1.715082e+09     0.000567                 21   
1515129  train  1.715083e+09     0.002496                 21   
1515324  train  1.715084e+09     0.001588                 21   
1515503  train  1.715085e+09     0.002269                 21   
1515733  train  1.715087e+09     0.002033                 20   
1515807  train  1.715088e+09     0.004448                 18   
1515981  train  1.715089e+09     0.002496                 21   
1516183  train  1.715090e+09     0.000567                 21   
1516391  train  1.715092e+09     0.001248                 21   
1516557  train  1.715092e+09     0.000000                 15   
1516832  train  1.715094e+09     0.002224                 21   
1516884  train  1.715095e+09     0.001588                 18   
1517097  train  1.715096e+09     0.003046                 21   
1517287  train  1.715097e+09     0.002295                 21   
1517496  train  1.715099e+09     0.000567                 21   
1517693  train  1.715100e+09     0.004550                 21   
1517858  train  1.715101e+09     1.773985                 19   
1518123  train  1.715102e+09     9.359838                 21   
1518226  train  1.715103e+09     9.21924

In [20]:
# Deconcat on source
train_separated = comb[comb['source'] == 'train'].copy()
test_separated = comb[comb['source'] == 'test'].copy()

In [21]:
# Making ID correspond to index
test_separated = test_separated.sort_values(by='ID').reset_index(drop=True)

In [22]:
# Drop source
test_separated.drop(columns=['source'], inplace=True)
train_separated.drop(columns=['source'], inplace=True)

In [23]:
test_separated.shape, train_separated.shape

((51739, 53), (1521790, 53))

<hr style="height:2px;border-width:0;color:blue;background-color:blue">

# TRAINING

In [24]:
# Choose X and y
X = [
    'year_rec', 'month_rec', 'day_rec', 'hour_rec', 'minute_rec', 'dayofweek_rec',
    # 'year_eta', 'month_eta', 'day_eta', 'hour_eta', 'minute_eta', 'dayofweek_eta', 'etaStd_seq',
    'time_seq', 'distance_km', 'time_diff_minutes', 'mph_last_stretch',
    'time_diff_minutes_lag1', 'latitude_lag1', 'longitude_lag1', 'mph_last_stretch_lag1', 'distance_km_lag1',
    'time_diff_minutes_lag2', 'latitude_lag2', 'longitude_lag2', 'mph_last_stretch_lag2', 'distance_km_lag2',
    'vesselId_encoded'
]
y = ['latitude', 'longitude']

# Scale train data frame
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = train_separated[X]
y_train = train_separated[y]

X_scaled = scaler_X.fit_transform(X_train)
y_scaled = scaler_y.fit_transform(y_train)

base_params = {
            'n_estimators': 100,
            'learning_rate': 0.1,
            'max_depth': 6,
            'objective': 'reg:squarederror'
        }

In [25]:
model = MultiOutputRegressor(xgb.XGBRegressor())
model.fit(X_scaled, y_scaled)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))

In [26]:
def update_lag_row(df, features_to_lag, index, steps=1):
    
    for step in range(1, steps + 1):
        
        for feature in features_to_lag:

            lag_col_name = f"{feature}_lag{step}"

            if index >= step and pd.isna(df.at[index, lag_col_name]):
                lagged_value = df.at[index - step, feature]

                # Only set the value if it's not NaN
                if not pd.isna(lagged_value):
                    df.at[index, lag_col_name] = lagged_value
    
    return df

In [48]:
test_df = test_separated[X + y + ['ID']]

In [ ]:
result_list = []
for index, row in test_df.iterrows():
    test_df = update_lag_row(test_df, features_to_lag, index, steps)
    
    # Set and scale input data
    input_data = row[X].to_frame().T  # Transpose to keep it as a single row DataFrame
    input_data_scaled = scaler_X.transform(input_data)
    
    # Predict and get original scale
    predicted_lat_long_scaled = model.predict(input_data)
    predicted_lat_long = scaler_y.inverse_transform(predicted_lat_long_scaled)
    
    predicted_lat = predicted_lat_long[0][0]  # Latitude prediction
    predicted_long = predicted_lat_long[0][1]  # Longitude prediction
    
    # Update DataFrame
    test_df.loc[index, 'latitude'] = predicted_lat
    test_df.loc[index, 'longitude'] = predicted_long
    
    #Append predictions to the result df
    result_list.append({
        'ID': index,
        'predicted_longitude': predicted_long,
        'predicted_latitude': predicted_lat
    })
    
    test_df.loc[index, 'distance_km'] = haversine(predicted_lat, predicted_long, 
                               test_df.loc[index, 'latitude_lag1'], test_df.loc[index, 'longitude_lag1'])
    test_df.loc[index, 'mph_last_stretch'] = (test_df.loc[index, 'distance_km'] *  0.621371) / (row['time_diff_minutes'] / 60)

In [ ]:
result = pd.DataFrame(result_list)

In [ ]:
result.head(1000)

In [ ]:
#result.to_csv('result_0.3.csv', index=False)